In [91]:
import cv2
import os
import numpy as np

def detect_aruco_markers_and_save(image_path, save_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Define the ArUco dictionary
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)
    
    # Define the parameters for marker detection
    parameters = cv2.aruco.DetectorParameters_create()
    
    # Detect markers in the image
    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    
    # Draw the detected markers on the image with markers in the bottom-left corner
    image_with_markers = cv2.aruco.drawDetectedMarkers(image.copy(), corners, ids, (0, 255, 0))
    
    # Get the image width and height
    height, width, _ = image.shape
    
    # Draw a red rectangle in the bottom-left corner
    
    # Save the image with markers and the red rectangle
    save_filename = os.path.join(save_path, 'detected_markers_image.jpg')
    cv2.imwrite(save_filename, image_with_markers)
    print(f"Image with detected markers saved at: {save_filename}")

    # Extract center points of each ID
    center_points = []
    for (marker, id) in zip(corners, ids):
        corner = marker.reshape((4, 2))
        (tl, tr, br, bl) = corner
        cx = int((tl[0] + br[0]) / 2)
        cy = int((tl[1] + br[1]) / 2)
        center_points.append((cx, cy))

        # Draw a blue circle at the center of each detected marker
        cv2.circle(image, (cx, cy), 4, (255, 255, 0), 2)  

        # Label the circle with marker ID and position
        label_text = f'ID:{id[0]} ({cx},{cy})'
        cv2.putText(image, label_text, (cx + 100, cy - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (1, 1, 1), 1, cv2.LINE_AA)

    # Create a separate image for the box with the same size as the original image
    center_points = np.array(center_points)
    hull = cv2.convexHull(center_points, clockwise=True, returnPoints=False)
    rect = cv2.minAreaRect(center_points)
    box = cv2.boxPoints(rect)
    box = np.int0(box)

    # Calculate area of the box
    area = cv2.contourArea(box)
    
    # Print pixel positions and area information
    print(f"Pixel Positions:")
    for point in center_points:
        print(f"({point[0]}, {point[1]})")

    print(f"Total Area of the Box: {area}")

    # Create an image for the box
    box_image = np.zeros_like(image)
    cv2.drawContours(box_image, [box], 0, (255, 0, 0), -1)

    # Blend the box image with the original image
    opacity = 0.25
    cv2.addWeighted(box_image, opacity, image, 1 - opacity, 0, image)
    
    save_filename_with_circles_and_box = os.path.join(save_path, 'image_with_circles_box.jpg')
    cv2.imwrite(save_filename_with_circles_and_box, image)
    print(f"Image with circles, blue box, and lines saved at: {save_filename_with_circles_and_box}")

# Example usage
image_path = '/home/qt/visionSort/visionSort_cv/Align.jpg'
save_path = '/home/qt/visionSort/visionSort_cv/imgs'
detect_aruco_markers_and_save(image_path, save_path)


Image with detected markers saved at: /home/qt/visionSort/visionSort_cv/imgs/detected_markers_image.jpg
Pixel Positions:
(206, 1348)
(970, 1335)
(967, 278)
(192, 274)
Total Area of the Box: 832504.0
Image with circles, blue box, and lines saved at: /home/qt/visionSort/visionSort_cv/imgs/image_with_circles_box.jpg
